# LUSID Performance Test

In this notebook we will be generating a portfolio with 10,000 holdings. Each holding will be the same instrument for simplicity, however it will be in a different sub-holding key and therefore will be a separate holding. Once we have loaded in 10,000 holdings, we will then run a query to retrieve these holdings.

In [25]:
# Import LUSID
import lusid as lu
import lusid.models as lm
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
)

from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

# Import Libraries
from datetime import datetime, timedelta, time
from IPython.display import display_html
import pytz
import pandas as pd
import matplotlib.pyplot as plt
import uuid
from lusidjam.refreshing_token import RefreshingToken
import math
import json
import os

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = lu.utilities.ApiClientFactory(
    token=RefreshingToken(), api_secrets_filename=secrets_path
)

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID SDK Version:  0.6.8980.0


## Set API's and variables

In [33]:
transaction_portfolios_api = api_factory.build(lu.api.TransactionPortfoliosApi)
property_definitions_api = api_factory.build(lu.api.PropertyDefinitionsApi)

In [1]:
scope = "performanceTest"
portfolio_code = "performanceTestPortfolio"

## Create Instrument

In [69]:
instruments_data = pd.DataFrame({'Local Currency Code': ['GBP'], 'Security Description': ['Lloyds Banking Group PLC'], 'Ticker':['LLOY LN']})
instruments_data

,Local Currency Code,Security Description,Ticker
0,GBP,Lloyds Banking Group PLC,LLOY LN


In [70]:
instrument_mapping_required = {
  'name': 'Security Description'
}

instrument_identifier_mapping = {
    'ClientInternal': 'Ticker'
}

response = load_from_data_frame(
    api_factory=api_factory, 
    data_frame=instruments_data, 
    identifier_mapping=instrument_identifier_mapping, 
    mapping_required=instrument_mapping_required,
    mapping_optional={},
    file_type='instruments',
    scope=scope)

succ, failed, errors = format_instruments_response(response)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


## Create SHK Property

In [71]:
try:
    property_definitions_api.create_property_definition(
        create_property_definition_request=lm.CreatePropertyDefinitionRequest(
            domain = "Transaction",
            scope = scope,
            code = "key_number",
            value_required = None,
            display_name = "key_number",
            data_type_id = lu.ResourceId(scope="system", code="string"),
            life_time = None
        )
    )

except lu.exceptions.ApiException as e:
    print(json.loads(e.body)["title"])

## Create Portfolio

In [72]:
try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="GBP",
            created="2022-01-01",
            sub_holding_keys=['Transaction/performanceTest/key_number'],
        ),
    )

except lu.ApiException as e:
    print(json.loads(e.body)["title"])

## Add Holdings

We first run the below code to generate a table with 10,000 transactions that will generate holdings.

In [73]:
transactions_data = pd.DataFrame(columns=['transaction_id', 'transaction_type', 'Ticker', 'transaction_date', 'settlement_date', 'units', 'transaction_price', 'total_consideration.amount', 'total_consideration.currency', 'key_number'])

for i in range(0,10000):
    transactions_data = transactions_data.append({
        'transaction_id': 'tx'+str(i), 
        'transaction_type': 'Buy',
        'Ticker': 'LLOY LN',
        'transaction_date': '07-01-2022', 
        'settlement_date': '10-01-2022', 
        'units': 1, 
        'transaction_price': 40, 
        'total_consideration.amount': 40, 
        'total_consideration.currency': 'GBP',
        'key_number': str(i)}
        , ignore_index=True)

transactions_data

,transaction_id,transaction_type,Ticker,transaction_date,settlement_date,units,transaction_price,total_consideration.amount,total_consideration.currency,key_number
0,tx0,Buy,LLOY LN,07-01-2022,10-01-2022,1,40,40,GBP,0
1,tx1,Buy,LLOY LN,07-01-2022,10-01-2022,1,40,40,GBP,1
2,tx2,Buy,LLOY LN,07-01-2022,10-01-2022,1,40,40,GBP,2
3,tx3,Buy,LLOY LN,07-01-2022,10-01-2022,1,40,40,GBP,3
4,tx4,Buy,LLOY LN,07-01-2022,10-01-2022,1,40,40,GBP,4
...,...,...,...,...,...,...,...,...,...,...
9995,tx9995,Buy,LLOY LN,07-01-2022,10-01-2022,1,40,40,GBP,9995
9996,tx9996,Buy,LLOY LN,07-01-2022,10-01-2022,1,40,40,GBP,9996
9997,tx9997,Buy,LLOY LN,07-01-2022,10-01-2022,1,40,40,GBP,9997
9998,tx9998,Buy,LLOY LN,07-01-2022,10-01-2022,1,40,40,GBP,9998


We then upload these holdings to LUSID.

In [74]:
transaction_field_mapping_required = {
    "code": "$performanceTestPortfolio",
    "transaction_id": "transaction_id",
    "type": 'transaction_type',
    "transaction_date": 'transaction_date',
    "settlement_date": 'settlement_date',
    "units": "units",
    "transaction_price.price": "transaction_price",
    "transaction_price.type": "$Price",
    "total_consideration.amount": "total_consideration.amount",
    "total_consideration.currency": "total_consideration.currency",
    "transaction_currency": "total_consideration.currency"
    }

transaction_field_mapping_optional = {}

transaction_identifier_mapping = {
  'ClientInternal': 'Ticker',
}

response = load_from_data_frame(
    api_factory=api_factory, 
    scope=scope, 
    data_frame=transactions_data,
    mapping_required=transaction_field_mapping_required, 
    mapping_optional=transaction_field_mapping_optional,
    identifier_mapping=transaction_identifier_mapping,
    property_columns=['key_number'], 
    file_type='transaction')

succ, errors = format_transactions_response(response)
pd.DataFrame(data=[{"success": len(succ), "errors": len(errors)}])

,success,errors
0,1,0


## Get Holdings

Once everything is uploaded, we can retrieve the latest holdings data from our portfolio.

In [76]:
holdings = transaction_portfolios_api.get_holdings(scope=scope, code=portfolio_code)
lusid_response_to_data_frame(holdings)

,instrument_scope,instrument_uid,sub_holding_keys.Transaction/performanceTest/key_number.key,sub_holding_keys.Transaction/performanceTest/key_number.value.label_value,properties.Holding/default/SourcePortfolioId.key,properties.Holding/default/SourcePortfolioId.value.label_value,properties.Holding/default/SourcePortfolioId.effective_from,properties.Holding/default/SourcePortfolioScope.key,properties.Holding/default/SourcePortfolioScope.value.label_value,properties.Holding/default/SourcePortfolioScope.effective_from,holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,currency
0,default,LUID_00003D6T,Transaction/performanceTest/key_number,0,Holding/default/SourcePortfolioId,performanceTestPortfolio,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,performanceTest,0001-01-01 00:00:00+00:00,P,1.0,1.0,40.0,GBP,40.0,GBP,GBP
1,default,LUID_00003D6T,Transaction/performanceTest/key_number,1,Holding/default/SourcePortfolioId,performanceTestPortfolio,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,performanceTest,0001-01-01 00:00:00+00:00,P,1.0,1.0,40.0,GBP,40.0,GBP,GBP
2,default,LUID_00003D6T,Transaction/performanceTest/key_number,2,Holding/default/SourcePortfolioId,performanceTestPortfolio,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,performanceTest,0001-01-01 00:00:00+00:00,P,1.0,1.0,40.0,GBP,40.0,GBP,GBP
3,default,LUID_00003D6T,Transaction/performanceTest/key_number,3,Holding/default/SourcePortfolioId,performanceTestPortfolio,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,performanceTest,0001-01-01 00:00:00+00:00,P,1.0,1.0,40.0,GBP,40.0,GBP,GBP
4,default,LUID_00003D6T,Transaction/performanceTest/key_number,4,Holding/default/SourcePortfolioId,performanceTestPortfolio,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,performanceTest,0001-01-01 00:00:00+00:00,P,1.0,1.0,40.0,GBP,40.0,GBP,GBP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,default,CCY_GBP,Transaction/performanceTest/key_number,9995,Holding/default/SourcePortfolioId,performanceTestPortfolio,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,performanceTest,0001-01-01 00:00:00+00:00,B,-40.0,-40.0,-40.0,GBP,-40.0,GBP,GBP
19996,default,CCY_GBP,Transaction/performanceTest/key_number,9996,Holding/default/SourcePortfolioId,performanceTestPortfolio,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,performanceTest,0001-01-01 00:00:00+00:00,B,-40.0,-40.0,-40.0,GBP,-40.0,GBP,GBP
19997,default,CCY_GBP,Transaction/performanceTest/key_number,9997,Holding/default/SourcePortfolioId,performanceTestPortfolio,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,performanceTest,0001-01-01 00:00:00+00:00,B,-40.0,-40.0,-40.0,GBP,-40.0,GBP,GBP
19998,default,CCY_GBP,Transaction/performanceTest/key_number,9998,Holding/default/SourcePortfolioId,performanceTestPortfolio,0001-01-01 00:00:00+00:00,Holding/default/SourcePortfolioScope,performanceTest,0001-01-01 00:00:00+00:00,B,-40.0,-40.0,-40.0,GBP,-40.0,GBP,GBP
